In [1]:
import pandas as pd
import numpy as np

import bs4
from urllib.request import urlopen
from bs4 import BeautifulSoup

from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
def pdfdownload(url,filename):
    import urllib.request
    pdf_path =url #"http://distcourts.tap.nic.in/hcorders/2014/arbappl/arbappl_132_2014.pdf"
    response = urllib.request.urlopen(url)    
    file = open(r"E:/pdffiles/PdfDocument Scarping2/"+filename+".pdf", 'wb')
    file.write(response.read())
    file.close()

In [3]:
def Main(FromDate,ToDate):
    driver=webdriver.Chrome(ChromeDriverManager().install())
    driver.maximize_window()
    driver.get("http://distcourts.tap.nic.in/judgmnt_ts/#")

    sleep(2)
    
    #/html/body/form/div/div[2]/div/div/div/div/div[2]/div/section/div/div[1]/div[5]/center/div/table/thead/tr[2]/td/input
    driver.find_element_by_xpath("/html/body/form/div/div[2]/div/div/div/div/div[1]/section/div[1]/div[1]/div[2]/ul/li[4]/a").click()
    
    #driver.find_element_by_id("txtPINCode").clear()
    driver.find_element_by_id("frmdto").send_keys(FromDate)
    #driver.find_element_by_xpath("/html/body/form/div/div[2]/div/div/div/div/div[2]/div/section/div/div[1]/div[5]/center/div/table/thead/tr[2]/td/input").click()
    driver.find_element_by_id("todto").send_keys(ToDate)

    driver.find_element_by_xpath("/html/body/form/div/div[2]/div/div/div/div/div[2]/div/section/div/div[1]/div[5]/center/div/table/thead/tr[4]/td/select/option[3]").click()
    sleep(2)

    driver.find_element_by_id("searchtwofour").click()

    sleep(5)
    page_source = driver.page_source
    soup1 = BeautifulSoup(page_source, 'lxml')
    type(soup1)
    bs4.BeautifulSoup
    title = soup1.title
    
    JUDGE_NAME_list=[]
    CASE_NUMBER_list=[]
    FILING_NUMBER_list=[]
    TITLE_list=[]
    DATE_OF_DECISION_list=[]
    Status_list=[]
    
    


    PDF_file_Table_tag=soup1.find_all("table",{"id":"inftable"})[0]

    tr_tag=PDF_file_Table_tag.find_all("tr")
    print(len(tr_tag))

    for i in range(2,6):
        td_tag1=tr_tag[i].find_all("td")[0]
        td_tag2=tr_tag[i].find_all("td")[1]
        td_tag3=tr_tag[i].find_all("td")[2]
        td_tag4=tr_tag[i].find_all("td")[3]
        td_tag5=tr_tag[i].find_all("td")[4]

        td_tag6=tr_tag[i].find_all("td")[5]
        JUDGE_NAME_list.append(td_tag1.text)
        CASE_NUMBER_list.append(td_tag2.text)
        FILING_NUMBER_list.append(td_tag3.text)
        TITLE_list.append(td_tag4.text)
        DATE_OF_DECISION_list.append(td_tag5.text)
        
        document=td_tag2.text
        print(document)
        try:
          
            a_tag_url=td_tag6.find("a")["href"]
            Status_list.append(a_tag_url)
            print(str(i-1)+")"+a_tag_url)
            pdfdownload(a_tag_url,document.replace("/"," "))
            
        except:
            Status_list.append("NaN")
            
            print("No Document")
            
    df = pd.DataFrame(data={"JUDGE_NAME:":JUDGE_NAME_list,"CASE_NUMBER":CASE_NUMBER_list,"FILING_NUMBER":FILING_NUMBER_list,"TITLE":TITLE_list,"DATE_OF_DECISION":DATE_OF_DECISION_list,"Status":Status_list})
    df.to_csv(r"E:\pdffiles\csvfiles\court.csv", sep=',',index=False)
    df.head(50)



Main("11-02-2021","12-02-2021")